In [ ]:
#| default_exp camera_talk
#| test: false

In [ ]:
#| export
#| eval: false

from __future__ import annotations

import asyncio, datetime as dt, random, logging, os, contextlib
from DataTalks.arena_agent import parking_q, history

from agents import Agent, Runner
from agents.mcp import MCPServerSse         

log = logging.getLogger(__name__)

MCP_URL   = os.getenv("MCP_URL",   "http://tools:9001/sse")
LOCATION  = os.getenv("LOCATION", "Helsinki,FI")   # city or "lat,lon"


In [ ]:
#| export
#| eval: false

async def parking_camera_loop() -> None:
    "Send a Gen-UI block every 60 s."
    while True:
        free_slots = random.randint(0, 120)
        stamp      = dt.datetime.now().strftime("%H:%M:%S")

        ui_block   = await _make_ui_block(free_slots, stamp)

        # one SSE `message`
        sse = "event: message\ndata: " + ui_block.replace("\n", "\ndata: ") + "\n\n"
        await parking_q.put(sse)
        log.debug("parking-loop bubble @ %s", stamp)

        await asyncio.sleep(180)


In [ ]:
#| export
#| eval: false

#from contextlib import asynccontextmanager

#@asynccontextmanager
#async def lifespan(app):
#    "Run the parking loop for the lifetime of the FastAPI app."
#    task = asyncio.create_task(parking_camera_loop(), name="parking-loop")
#    yield
#    task.cancel()
#    with contextlib.suppress(asyncio.CancelledError):
#        await task


In [ ]:
#| export
#| eval: false

async def _make_ui_block(free_slots: int, stamp: str) -> str:
    """
    Generates a full GenUI HTML card via LLM using weather + parking.
    Ensures booking buttons and branding are always present.
    """
    async with MCPServerSse(name="ui", params={"url": MCP_URL}, client_session_timeout_seconds=300) as srv:
        
        instructions = f"""
        🎨 You are EcoGen, a generative-UI designer.
        
        You will craft **one complete MonsterUI/Tailwind card** and stream it via HTMX *exactly once*.
        
        Inputs this run
        • 🅿️  {free_slots} free parking slots
        • 🕒  Current time: {stamp}
        • 🌤️  Live weather for “{LOCATION}”
        
        Step‑by‑step
        1. **Call `weather.current` once.**  Parse `temperature` (°C), `windspeed` (m/s) and `description`.
        
        2. **Call `ImageGenerationTool` once** with a tiny prompt such as “bright sun on white background”.  When the JSON returns, extract
           • `url`  – image URL  
           and put it straight into an `<img>` tag – **no {{curly}} placeholders**:
        
           ```html
           <img class="rounded-lg object-cover w-32 h-32 float-right ml-4"
                src="https://…real-url…" alt="bright sun on white background">
           ```
        
        3. **Compose one HTML block** wrapped in
        
           ```html
           <div hx-swap-oob="beforeend:#chatlog">
             …full card…
           </div>
           ```
        
           It must contain, in order:
           • A fun heading with an emoji  
           • Weather summary (temp + wind + description + the image)  
           • Parking status line, e.g. `🚗 40 free slots`  
           • A friendly encouragement paragraph  
           • **Exactly three** booking buttons, styled `btn btn-success btn-sm`, linking to:
             – City Bikes (HSL) → https://kaupunkipyorat.hsl.fi/en  
             – Donkey Republic → https://app.donkeyrepublic.com/#/reserve  
             – ListNRide → https://www.listnride.com/espoo  
             Place them inside
        
           ```html
           <div class="grid grid-cols-1 sm:grid-cols-3 gap-2 mt-4">…</div>
           ```
        
           • Footer line “Wheel be seeing you! 🌱”  
           • Branding footer
        
           ```html
           <div class="text-xs text-center text-gray-400 mt-3">Powered by EventTalks 🏒🌍</div>
           ```
        
        Rules
        • **HTML only** – no Markdown, no JSON.  
        • Produce a single response; do **not** send drafts.  
        • Missing any mandatory element ⇒ response rejected.
        """
        agent = Agent(
            name="EcoGen Agent",
            model="o3",
            mcp_servers=[srv],
            
            instructions=instructions,
        )

        res = await Runner.run(agent, input="")
        ui = res.final_output.strip()

    # Safety net: wrap if LLM forgot
    if "hx-swap-oob" not in ui:
        ui = f'<div hx-swap-oob="beforeend:#chatlog">{ui}</div>'

    # Safety net: inject buttons if missing
    if all(x not in ui for x in ["City Bikes", "Donkey Republic", "ListNRide"]):
        ui += """
        <div class="grid grid-cols-1 sm:grid-cols-3 gap-2 mt-4">
          <a href="https://kaupunkipyorat.hsl.fi/en"               class="btn btn-success btn-sm">🚲 City Bikes (HSL)</a>
      <a href="https://app.donkeyrepublic.com/#/reserve"       class="btn btn-success btn-sm">🦄 Donkey Republic</a>
     <a href="https://www.listnride.com/espoo"                class="btn btn-success btn-sm">🚴 ListNRide</a>
            </div>
        """

    # Branding if missing
    if "EventTalks" not in ui:
        ui += '<div class="text-xs text-center text-gray-400 mt-3">Powered by EventTalks 🏒🌍</div>'

    history.append(ui)
    return ui



